# POC of Backend Flow for Pokeradar

## Imports

In [2]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

In [3]:
def initialize_driver():
    # Initialize the driver (ensure ChromeDriver is in your system's PATH)
    driver = webdriver.Chrome()
    return driver

def click_state_link(driver, state):
    # Open the page with the state as a parameter
    url = "https://support.pokemoncenter.com/hc/en-us/sections/13360842288916-Pok%C3%A9mon-Automated-Retail-Vending-Machines"
    driver.get(url)
    
    # Wait for the page to load
    time.sleep(3)
    
    # Find the link based on the state (handle both full state name and abbreviation)
    state_link_xpath = f"//a[contains(text(), '{state} Pokémon Automated Retail Vending Machine Locations')]"
    
    try:
        # Locate and click the link
        state_link = driver.find_element(By.XPATH, state_link_xpath)
        state_link.click()
        print(f"Clicked on {state} link")
    except Exception as e:
        print(f"Error: {e}")
        print(f"Could not find link for {state}")

    # Wait for the redirected page to load
    time.sleep(3)

def extract_table(driver):
    # Find the table element
    table = driver.find_element(By.XPATH, "//table")
    
    # Get all rows from the table
    rows = table.find_elements(By.TAG_NAME, "tr")
    
    # Extract table headers
    headers = [header.text for header in rows[0].find_elements(By.TAG_NAME, "th")]
    
    # Extract table data
    table_data = []
    for row in rows[1:]:  # Skip the header row
        cells = row.find_elements(By.TAG_NAME, "td")
        row_data = [cell.text for cell in cells]
        table_data.append(row_data)
    
    # Convert to DataFrame using Pandas
    df = pd.DataFrame(table_data, columns=headers)
    
    return df


In [4]:
state = "California"

In [5]:
driver = initialize_driver()
click_state_link(driver, state)
df = extract_table(driver)
time.sleep(5)

Clicked on California link


In [6]:
# Close the browser after completion
driver.quit()

In [7]:
df

,Retailer,Machine ID,Address,"City, State"
0,Safeway,Q01036,2227 S Shore Center,"Alameda, CA"
1,Safeway,Q01268,2600 5th St,"Alameda, CA"
2,Food 4 Less,Q00330,1616 W Katella Ave,"Anaheim, CA"
3,Albertsons,Q00491,810 S State College Blvd,"Anaheim, CA"
4,Vons,Q00695,5600 Santa Ana Canyon Rd,"Anaheim, CA"
...,...,...,...,...
239,Pavilions,Q00415,8969 Santa Monica Blvd,"West Hollywood, CA"
240,Albertsons,Q00662,6755 Westminster Blvd,"Westminster, CA"
241,Vons,Q00327,15740 La Forge St,"Whittier, CA"
242,Vons,Q01240,20445 Yorba Linda Blvd,"Yorba Linda, CA"


In [30]:
df_imputed = df.copy()
df_imputed['Address'] = df_imputed.apply(lambda row: row['Address'] + ', ' + row['City, State'], axis=1)
df_imputed[['City', 'State']] = df_imputed['City, State'].apply(lambda x: pd.Series(x.split(', ')))
df_imputed.drop(columns=['City, State'], inplace=True)
df_imputed

,Retailer,Machine ID,Address,City,State
0,Safeway,Q01036,"2227 S Shore Center, Alameda, CA",Alameda,CA
1,Safeway,Q01268,"2600 5th St, Alameda, CA",Alameda,CA
2,Food 4 Less,Q00330,"1616 W Katella Ave, Anaheim, CA",Anaheim,CA
3,Albertsons,Q00491,"810 S State College Blvd, Anaheim, CA",Anaheim,CA
4,Vons,Q00695,"5600 Santa Ana Canyon Rd, Anaheim, CA",Anaheim,CA
...,...,...,...,...,...
239,Pavilions,Q00415,"8969 Santa Monica Blvd, West Hollywood, CA",West Hollywood,CA
240,Albertsons,Q00662,"6755 Westminster Blvd, Westminster, CA",Westminster,CA
241,Vons,Q00327,"15740 La Forge St, Whittier, CA",Whittier,CA
242,Vons,Q01240,"20445 Yorba Linda Blvd, Yorba Linda, CA",Yorba Linda,CA


In [9]:
from geopy.geocoders import Nominatim
import folium

In [10]:
geolocator = Nominatim(user_agent="geoapi")
locations = {}

In [ ]:
for address in df_imputed['Address']:
    if address not in set(locations.keys()):
        location = geolocator.geocode(address)
        locations[address] = location
        time.sleep(2)
print(locations)

{'2227 S Shore Center, Alameda, CA': Location(Safeway, 2227, South Shore Center, South Shore, Alameda, Alameda County, California, 94501, United States, (37.7574333, -122.2507884, 0.0)), '2600 5th St, Alameda, CA': Location(Safeway, 2600, 5th Street, Jack London Square, Downtown Oakland, Oakland, Alameda County, California, 94501, United States, (37.78553005, -122.28057335977067, 0.0)), '1616 W Katella Ave, Anaheim, CA': Location(1616, West Katella Avenue, Neff, Anaheim, Orange County, California, 92802, United States, (33.803257548387094, -117.93901661290323, 0.0)), '810 S State College Blvd, Anaheim, CA': Location(Albertsons, 810, South State College Boulevard, Anaheim, Orange County, California, 92806, United States, (33.8313152, -117.8886498, 0.0)), '5600 Santa Ana Canyon Rd, Anaheim, CA': Location(Vons, 5600, Santa Ana Canyon Road, Anaheim Hills, Anaheim, Orange County, California, 92807, United States, (33.849789799999996, -117.79166178558157, 0.0)), '16 Rancho del Mar, Aptos, CA

In [18]:
len(locations.keys())

244

In [31]:
geo_df = pd.DataFrame(list(locations.items()), columns=['Address', 'Geocode'])

In [32]:
geo_df

,Address,Geocode
0,"2227 S Shore Center, Alameda, CA","(Safeway, 2227, South Shore Center, South Shor..."
1,"2600 5th St, Alameda, CA","(Safeway, 2600, 5th Street, Jack London Square..."
2,"1616 W Katella Ave, Anaheim, CA","(1616, West Katella Avenue, Neff, Anaheim, Ora..."
3,"810 S State College Blvd, Anaheim, CA","(Albertsons, 810, South State College Boulevar..."
4,"5600 Santa Ana Canyon Rd, Anaheim, CA","(Vons, 5600, Santa Ana Canyon Road, Anaheim Hi..."
...,...,...
239,"8969 Santa Monica Blvd, West Hollywood, CA","(Pavillions, 8969, Santa Monica Boulevard, Wes..."
240,"6755 Westminster Blvd, Westminster, CA","(Albertsons, 6755, Westminster Boulevard, West..."
241,"15740 La Forge St, Whittier, CA","(La Forge Street, East Whittier, Whittier, Los..."
242,"20445 Yorba Linda Blvd, Yorba Linda, CA","(Vons, 20445, Yorba Linda Boulevard, Yorba Lin..."


In [33]:
df_imputed = pd.concat([df_imputed, geo_df[['Geocode']]], axis=1)

In [34]:
df_imputed

,Retailer,Machine ID,Address,City,State,Geocode
0,Safeway,Q01036,"2227 S Shore Center, Alameda, CA",Alameda,CA,"(Safeway, 2227, South Shore Center, South Shor..."
1,Safeway,Q01268,"2600 5th St, Alameda, CA",Alameda,CA,"(Safeway, 2600, 5th Street, Jack London Square..."
2,Food 4 Less,Q00330,"1616 W Katella Ave, Anaheim, CA",Anaheim,CA,"(1616, West Katella Avenue, Neff, Anaheim, Ora..."
3,Albertsons,Q00491,"810 S State College Blvd, Anaheim, CA",Anaheim,CA,"(Albertsons, 810, South State College Boulevar..."
4,Vons,Q00695,"5600 Santa Ana Canyon Rd, Anaheim, CA",Anaheim,CA,"(Vons, 5600, Santa Ana Canyon Road, Anaheim Hi..."
...,...,...,...,...,...,...
239,Pavilions,Q00415,"8969 Santa Monica Blvd, West Hollywood, CA",West Hollywood,CA,"(Pavillions, 8969, Santa Monica Boulevard, Wes..."
240,Albertsons,Q00662,"6755 Westminster Blvd, Westminster, CA",Westminster,CA,"(Albertsons, 6755, Westminster Boulevard, West..."
241,Vons,Q00327,"15740 La Forge St, Whittier, CA",Whittier,CA,"(La Forge Street, East Whittier, Whittier, Los..."
242,Vons,Q01240,"20445 Yorba Linda Blvd, Yorba Linda, CA",Yorba Linda,CA,"(Vons, 20445, Yorba Linda Boulevard, Yorba Lin..."


In [36]:
import folium
import ast  # To safely convert string tuples to actual tuples

# Initialize Folium Map (centered around California)
m = folium.Map(location=[37.7749, -122.4194], zoom_start=7)
# Loop through the DataFrame
for _, row in df_imputed.iterrows():
    try:
        latitude, longitude = row['Geocode'].latitude, row['Geocode'].longitude
        
        # Construct detailed popup with additional metadata
        popup_text = (
            f"<b>Retailer:</b> {row['Retailer']}<br>"
            f"<b>Machine ID:</b> {row['Machine ID']}<br>"
            f"<b>Address:</b> {row['Address']}<br>"
            f"<b>City:</b> {row['City']}<br>"
            f"<b>State:</b> {row['State']}"
        )
        
        # Add marker with metadata
        folium.Marker(
            [latitude, longitude], 
            popup=folium.Popup(popup_text, max_width=300)
        ).add_to(m)
    
    except Exception as e:
        print(f"Error processing row {row['Address']}: {e}")
display(m)


Error processing row 2550 Bell Rd, Auburn, CA: 'NoneType' object has no attribute 'latitude'
Error processing row 11800 De Palma Rd, Corona, CA: 'NoneType' object has no attribute 'latitude'
Error processing row 311 W Los Feliz Rd, Glendale, CA: 'NoneType' object has no attribute 'latitude'
Error processing row 27702 Crown Valley Pkwy Suite B, Ladera Ranch, CA: 'NoneType' object has no attribute 'latitude'
Error processing row 4365 Glencoe Ave, Marina del Rey, CA: 'NoneType' object has no attribute 'latitude'
Error processing row 23850 Copperhill Dr, Valencia, CA: 'NoneType' object has no attribute 'latitude'


In [17]:
df_imputed.to_csv('../data/data.csv')

In [39]:
m.save("../map/map.html")